In [ ]:
!pip install -q "tensorflow-text==2.8.*"
!pip install tokenizers
!pip install rouge-metric

!pip install pyunpack
!pip install patool
!pip install -q tf-models-official==2.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.1/498.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 55.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.19.6

In [ ]:
from pyunpack import Archive
Archive('/content/drive/MyDrive/arxiv-dataset.zip').extractall('/content')

In [ ]:
import nltk
import json
import re
import time
import sys
import os
import shutil
import gc # garbage collector
import keras
import tensorflow as tf
import numpy as np
import tensorflow_text as text
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import TextVectorization, LSTM, Bidirectional, Embedding, Input, Attention, TimeDistributed, Reshape
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize, sent_tokenize
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
from rouge_metric import PyRouge

tf.config.run_functions_eagerly(True)

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

RuntimeError: ignored

RuntimeError: ignored

RuntimeError: ignored

RuntimeError: ignored

In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
vocab_size = 40000

tokenizer = Tokenizer(BPE())
trainer = BpeTrainer(limit_alphabet=vocab_size)
tokenizer.pre_tokenizer = Whitespace()

tokenizer.train(['/content/arxiv-dataset/vocab'], trainer)

tokenizer.enable_padding()
vectorizer = tokenizer

In [ ]:
# удаляет пунктуацию, переводит в нижний регистр
def normalizeString(s):
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
def build_gt_summary2(doc, article, summary):

  highest_score = 0
  rouge_L = text.metrics.rouge_l

  article = tf.RaggedTensor.from_tensor(article)

  indicies = np.empty((summary.shape[0]*2,)) # индексы предложений с самым высоким rouge
  i = 0 # индексы индексов (чтобы добавлять новый элемент в indicies на i-ю позицию)

  # для отладки (считаем количество предложений в summary)
  count = 0

  # для кажого предложения из summary ищем предложение из article
  for summary_sent in summary:
    if not (summary_sent == 0).all():
      count += 1

    # создаем массив из повторяющегося summary_sent для параллельного вычисления
    summary_sent = summary_sent.reshape((1, summary.shape[1]))
    summary_sent_rep = np.repeat(summary_sent, article.shape[0], axis=0)
    summary_sent_rep = tf.RaggedTensor.from_tensor(summary_sent_rep)

    # вычисляем rouge между summary_sent и каждым предложением из article одновременно
    scores, _, _ = rouge_L(summary_sent_rep, article)

    # находим индексы
    max_idx1 = np.argmax(scores) # индекс предложения с самым высоким rouge
    top_2_scores = np.concatenate((scores[:max_idx1], scores[max_idx1+1:])) # значения rouge без самого высокого
    max_idx2 = np.argmax(top_2_scores)

    # добавляем индексы
    indicies[i] = max_idx1
    i += 1
    indicies[i] = max_idx2
    i += 1

  # бинарная маска is_summary_mask[i] = 1 если article[i] принадлежит эктрактивному реферату
  #is_summary_mask = np.zeros((article.shape[0],))
  #is_summary_mask[indicies.astype('int32')] = 1

  return indicies #is_summary_mask

In [ ]:
# return indicies of tokens in article
def preproc_doc(doc, vectorizer, max_doc_len, max_sent_len):

    doc = bytes.decode(doc.numpy())
    doc = json.loads(doc)

    article_text = np.asarray( [ normalizeString(sentence) for sentence in doc["article_text"] ] )

    article_indicies = [tokens.ids for tokens in vectorizer.encode_batch(article_text)]

    doc_len = len(article_indicies)

    sent_padding_tensor = np.empty((max_sent_len,))
    seq = article_indicies + [sent_padding_tensor]
    padded_sents = pad_sequences(seq, maxlen=max_sent_len, padding='post', truncating='post')[:-1]

    doc_padding_tensor = np.zeros((max_doc_len, max_sent_len))
    doc_padding_tensor = tf.constant(doc_padding_tensor)

    seq = [padded_sents, doc_padding_tensor]
    padded_doc = pad_sequences(seq, maxlen=max_doc_len, padding='post', truncating='post')[0][:,:max_sent_len]

    return padded_doc

In [ ]:
# return indicies of ground-truth summary sentences
def preproc_abs(doc, vectorizer, max_abst_len, max_sent_len):

    doc = bytes.decode(doc.numpy())
    doc = json.loads(doc)

    abstract_text = np.asarray( [ normalizeString(sentence) for sentence in doc["abstract_text"] ] )

    abstract_indicies = [tokens.ids for tokens in vectorizer.encode_batch(abstract_text)]

    abs_len = len(abstract_indicies)

    sent_padding_tensor = np.empty((max_sent_len,))
    seq = abstract_indicies + [sent_padding_tensor]
    padded_sents = pad_sequences(seq, maxlen=max_sent_len, padding='post', truncating='post')[:-1]

    abs_padding_tensor = np.zeros((max_abst_len, max_sent_len))
    abs_padding_tensor = tf.constant(abs_padding_tensor)

    seq = [padded_sents, abs_padding_tensor]
    padded_abs = pad_sequences(seq, maxlen=max_abst_len, padding='post', truncating='post')[0][:,:max_sent_len]

    return padded_abs

In [ ]:
def gen_train_data(x, vectorizer, max_doc_len, max_abst_len, max_sent_len):
    # generate tuple (document, decoder_inputs, expected_decoder_outputs)
    document = preproc_doc(x, vectorizer, max_doc_len, max_sent_len)
    summary = preproc_abs(x, vectorizer, max_abst_len, max_sent_len)

    gt_summary = build_gt_summary2(x, document, summary)

    return document, gt_summary

def make_ds(vectorizer, stop_words, file_path="train.txt", max_doc_len=300, max_abst_len=90, max_sent_len=35, batch_size=32):
    gen_train_data_ = lambda x: gen_train_data(x, vectorizer, max_doc_len, max_abst_len, max_sent_len)

    arXiv_ds = tf.data.TextLineDataset(file_path).map((lambda x: tf.py_function(gen_train_data_, [x], [tf.int64, tf.int64]))).batch(batch_size, drop_remainder=True)

    return arXiv_ds

In [ ]:
max_doc_len = 250
max_abst_len = 15
max_sent_len = 30
batch_size = 8

train_src = "/content/arxiv-dataset/train.txt"
train_dst = "/content/train.txt"

val_src = "/content/arxiv-dataset/val.txt"
val_dst = "/content/val.txt"

test_src = "/content/arxiv-dataset/test.txt"
test_dst = "/content/test.txt"

shutil.move(train_src, train_dst)
shutil.move(val_src, val_dst)
shutil.move(test_src, test_dst)

'/content/test.txt'

In [ ]:
validation_data = make_ds(vectorizer, "val.txt", max_doc_len=max_doc_len, max_abst_len=max_abst_len, max_sent_len=max_sent_len, batch_size=batch_size)
arXiv_ds = make_ds(vectorizer, "train.txt", max_doc_len=max_doc_len, max_abst_len=max_abst_len, max_sent_len=max_sent_len, batch_size=batch_size)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [ ]:
start = time.time()
for ex in arXiv_ds.take(1):
    print("document:")
    print(ex[0][1])
    print()
    print("expected decoder output:")
    print(ex[1][1])


end = time.time()
print(end-start)

document:
tf.Tensor(
[[  830 16834  9308 ...  1983  1841   187]
 [  830 16834  2471 ...   224   830  7696]
 [  830  1656   360 ...  7048   459  5826]
 ...
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]], shape=(250, 30), dtype=int64)

expected decoder output:
tf.Tensor(
[ 22  39  36  49 139 139 139 139 139 139 139 139 139 139 139 139 139 139
 139 139 139 139 139 139 139 139 139 139 139 139], shape=(30,), dtype=int64)
1.927180528640747


In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, embeddings_dim=300, vocab_size=40000, bilstm_units=256, lstm_units=512, batch_size=32):
    super(Encoder, self).__init__()

    self.embeddings_dim = embeddings_dim
    self.vocab_size = vocab_size
    self.bilstm_units = bilstm_units
    self.lstm_units = lstm_units
    self.batch_size = batch_size

    """
    token_level RNN produce sentence embeddigs
    input.shape = (max_doc_length, max_sentence_length, 300)
    300 - embeddings size
    output.shape = (max_doc_length, 512)
    """
    self.embedding = Embedding(self.vocab_size, self.embeddings_dim, mask_zero=True, name='encoder_word_embedding')
    self.token_level = TimeDistributed(Bidirectional(LSTM(self.bilstm_units, dropout=0.5, name='encoder_token_level_rnn')))

    """
    sentence_level RNN produce contexual document representation
    input = [[s1, s2, .. si] for i in [1..max_doc_length]], si - i-th sentence
    input.shape = (max_doc_length, max_sent_length, 512)
    output.shape = (max_doc_length, 512)
    """

    self.sentence_level = TimeDistributed(Bidirectional(LSTM(self.bilstm_units, dropout=0.5, name='encoder_sentence_level_rnn')))

  #@tf.function(input_signature=[tf.TensorSpec([None, max_doc_len, max_sent_len], tf.int64)])
  def call(self, batch=None):
    '''batch - tensor of shape [32, max_doc_length, max_sent_length]'''

    batch_size = batch.shape[0]
    max_doc_length = batch.shape[1]

    # tensor to mask contexts for sentence-level LSTM
    mask = tf.sequence_mask(range(1, max_doc_length + 1), max_doc_length)
    mask = tf.expand_dims(mask, 0)
    if not (batch_size is None):
      mask = tf.tile(mask, [batch_size, 1, 1])

    # maximum timesteps for decoder's autoregression
    n_timesteps = 180

    # encoder output:

    # sentence embeddings
    # S.shape = (batch_size, max_doc_length, 512)
    embeddings = self.embedding(batch)

    S = self.token_level(embeddings)

    # document representations, D.shape = (batch_size, max_doc_length, max_doc_length, lstm_units)
    S_with_new_axis = tf.expand_dims(S, 1)
    contexts = tf.tile(S_with_new_axis, [1, max_doc_length, 1, 1])
    contexts = tf.ragged.boolean_mask(contexts, mask)
    contexts = contexts.to_tensor(default_value=0.0)

    D = self.sentence_level(contexts)

    del contexts
    del mask

    return D, S

In [ ]:
embeddings_dim = 48
lstm_units = 48
bilstm_units = 24

test_Encoder = Encoder(embeddings_dim=embeddings_dim, bilstm_units=bilstm_units, lstm_units=lstm_units, batch_size=batch_size)

for example in arXiv_ds.take(1):
  test_encoder_input = example[0]

test_sent_contxt, test_sent_emb = test_Encoder(test_encoder_input)

print(f"sentence embeddings shape: (batch_size, max_doc_length, lstm_units) {test_sent_emb.shape}")
print(f"contextual sentence representations shape: (batch_size, max_doc_length, lstm_units) {test_sent_contxt.shape}")

sentence embeddings shape: (batch_size, max_doc_length, lstm_units) (8, 250, 48)
contextual sentence representations shape: (batch_size, max_doc_length, lstm_units) (8, 250, 48)


In [ ]:
class LuongAttention(tf.keras.layers.Layer):
  def __init__(self):
    super(LuongAttention, self).__init__()
    #self.W1 = tf.keras.layers.Dense(units)
    #self.W2 = tf.keras.layers.Dense(units)
    #self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    query = query.astype('float32')
    values = values.astype('float32')
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    values_transposed = tf.transpose(values, perm=[0, 2, 1])

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    #BAHDANAU ADDITIVE:
    #score = self.V(tf.nn.tanh(
    #self.W1(query_with_time_axis) + self.W2(values)))

    #LUONGH Dot-product
    score = tf.transpose(tf.matmul(query_with_time_axis, values_transposed), perm=[0, 2, 1])

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, tf.squeeze(attention_weights)

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, bilstm_units=256, lstm_units=512, batch_size=32):
    super(Decoder, self).__init__()

    self.bilstm_units = bilstm_units
    self.lstm_units = lstm_units
    self.batch_size = batch_size

    # set the decoder_lstm to proccess one timestep at time
    self.decoder_lstm = LSTM(self.lstm_units, dropout=0.5, return_sequences=True, return_state=True, name='decoder_rnn')

    self.attention = LuongAttention()

  def build(self, input_shape):
    # matrix to produce attention state
    self.W_h = self.add_weight(name="attention_state_weights_0", shape=(input_shape[0], 1, 2), initializer="random_normal", trainable=True)

  def call(self, document_representations, sentence_embeddings, decoder_input, initial_state=None):
    # decoder_input - input of the current step
    # initial_state - list of previos decoder states [decoder_h, decoder_c]
    # document_representations contectual representations by the encoder
    D = document_representations
    out, decoder_h, decoder_c = self.decoder_lstm(decoder_input, initial_state=initial_state)
    decoder_last_state_h = decoder_h

    # attention weights are used as output probability distribution over the document sentences, of the choice for the next extracted sentence
    context_vector, attention_weights = self.attention(decoder_last_state_h, D)

    context_vector = tf.expand_dims(context_vector, axis=1)
    decoder_last_state_h = tf.expand_dims(decoder_last_state_h, 1)

    attention_state = tf.matmul(self.W_h, tf.concat([context_vector, decoder_last_state_h], axis=1))
    attention_state_T = tf.transpose(attention_state, perm=[0, 2, 1])

    # indicies of the extracted sentences
    indicies = tf.math.argmax(attention_weights, axis=1)
    flatten_indicies = tf.reshape(indicies, [-1])
    extracted_sentences = sentence_embeddings[np.arange(batch_size), flatten_indicies]
    extracted_sentences = tf.expand_dims(extracted_sentences, axis=1)
    extracted_sentences_T = tf.transpose(extracted_sentences, perm=[0, 2, 1])

    # input for next decoder step
    decoder_input_T = tf.concat([extracted_sentences_T, attention_state_T], axis=2)
    decoder_input = tf.transpose(decoder_input_T, perm=[0, 2, 1])

    initial_state = [decoder_h, decoder_c]

    return attention_weights, decoder_input, [decoder_h, decoder_c]

In [ ]:
test_Decoder = Decoder(bilstm_units=bilstm_units, lstm_units=lstm_units, batch_size=batch_size)

initial_input = tf.zeros((batch_size, lstm_units))
initial_input = tf.expand_dims(initial_input, 1)

attention_weights, decoder_input, states = test_Decoder(test_sent_contxt, test_sent_emb, initial_input)

print(f"probability distribution of sentences belonging summary shape: (batch_size, max_doc_length, 1) {attention_weights.shape}")
print(f"decoder input for the next step shape: {decoder_input.shape}")
print(f"decoder states for the next step shape: {states[0].shape, states[1].shape}")

probability distribution of sentences belonging summary shape: (batch_size, max_doc_length, 1) (8, 250)
decoder input for the next step shape: (8, 2, 48)
decoder states for the next step shape: (TensorShape([8, 48]), TensorShape([8, 48]))


In [ ]:
class Pointer(keras.Model):
  def __init__(self, embeddings_dim=300, vocab_size=40000, bilstm_units=256, lstm_units=512, batch_size=32, max_timesteps=20):
    super(Pointer, self).__init__()

    self.embeddings_dim = embeddings_dim
    self.vocab_size = vocab_size
    self.bilstm_units = bilstm_units
    self.lstm_units = lstm_units
    self.batch_size = batch_size

    self.encoder = Encoder(embeddings_dim=self.embeddings_dim,
                           vocab_size=self.vocab_size,
                           bilstm_units=self.bilstm_units,
                           lstm_units=self.lstm_units,
                           batch_size=self.batch_size)

    self.decoder = Decoder(bilstm_units=self.bilstm_units,
                           lstm_units=self.lstm_units,
                           batch_size=self.batch_size)

    # max length of sequence, genereted by decoder
    self.max_timesteps = max_timesteps

  def train_step(self, batch, partial_out=True):
    # partial_out=True если нужно применять модель к части входных данных, генерировать частичный ответ, потом соединять все в итоговый
    document, summary_indicies = batch
    #summary_indicies = summary_indicies.astype('float32')

    with tf.GradientTape() as tape:
      prob_distributions = self(document)

      loss = self.compiled_loss(summary_indicies, prob_distributions)
      training_vars = self.trainable_variables

      gradients = tape.gradient(loss, training_vars)

      self.optimizer.apply_gradients(zip(gradients, training_vars))
      self.compiled_metrics.update_state(summary_indicies, document)

    del tape
    return {m.name : m.result() for m in self.metrics}


  def call(self, batch, initial_decoder_states=None, training=True):
    batch_size = batch.shape[0]
    max_doc_length = batch.shape[1]

    # set decoder to process one timestep at time
    initial_input = tf.zeros((self.batch_size, self.lstm_units))
    initial_input = tf.expand_dims(initial_input, 1)
    decoder_input = initial_input

    document_representations, sentence_embeddings = self.encoder(batch)
    probability_distributions = []

    if training:

      states = initial_decoder_states

      for i in range(self.max_timesteps):
        attention_weights, decoder_input, states = self.decoder(document_representations, sentence_embeddings, decoder_input, states)
        probability_distributions.append(attention_weights)

      decoder_outputs = tf.stack(probability_distributions, axis=1)
      return decoder_outputs

    else:
      states = initial_decoder_states

      for i in range(self.max_timesteps):
        attention_weights, decoder_input, states = self.decoder(document_representations, sentence_embeddings, decoder_input, states)
        probability_distributions.append(attention_weights)

      return tf.constant(probability_distributions)

In [ ]:
checkpoint_path = '/content/drive/MyDrive/model_weights.cpkt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True,
                                                 verbose=1)

imported_model = tf.keras.models.load_model(checkpoint_path)

start = time.time()

class MyCustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
      tf.keras.backend.clear_session()
      gc.collect()

gc_callback = MyCustomCallback()

probability_distributions = imported_model.predict(arXiv_ds.take(1))
print(f"probability distributions of the 1st document shape: {probability_distributions.shape}")

print(f"time in sec: {time.time() - start}")

probability distributions of the 1st document shape: (8, 30, 250)
time in sec: 9.068718194961548


In [ ]:
def preproc_text_ds(doc):
    doc = bytes.decode(doc.numpy())
    doc = json.loads(doc)

    doc = doc["article_text"]

    if 250 - len(doc) > 0:
      paddings = [[0, 250 - len(doc)]]
      doc = tf.pad(doc, paddings, constant_values="")
    else:
      doc = doc[:250]

    return np.array(doc)

def get_sections_text(doc):
    doc = bytes.decode(doc.numpy())
    doc = json.loads(doc)

    sections = doc["sections"]

    padded_sections = []

    for ids in sections:

      if 250 - len(ids) > 0:
        paddings = [[0, 250 - len(ids)]]
        ids = tf.pad(ids, paddings, constant_values="")
      else:
        ids = ids[:250]

      padded_sections.append(ids)

    return np.array(padded_sections)

def get_sections(doc):
    doc = bytes.decode(doc.numpy())
    doc = json.loads(doc)

    sections = doc["sections"]

    indicies = [[tokens.ids for tokens in vectorizer.encode_batch(section)] for section in sections]

    padded_indicies = []
    for ids in indicies:
      ids_padded = []
      if 30 - len(ids[0]) < 0:
        for id in ids:
         ids_padded.append(id[:30])
      else:
        ids_padded = ids


      if 250 - len(ids_padded) > 0:
        paddings = [[0, 250 - len(ids_padded)], [0, 30-len(ids_padded[0])]]
        ids_padded = tf.pad(ids_padded, paddings, constant_values=0)
      else:
        ids_padded = ids_padded[:250]

      padded_indicies.append(ids_padded)

    return np.array(padded_indicies)

In [ ]:
def genereate_partial_abs(sections, model):
  if sections.shape[0] > 8:
    sections = sections[:8]
  else:
    sections = tf.pad(sections, [[0, 8 - sections.shape[0]], [0, 0], [0, 0]])

  sections_abs_logits = tf.math.log(model.predict(sections))

  sequence_lengths = tf.fill([batch_size], 6)
  decoded, probs = tf.nn.ctc_beam_search_decoder(tf.transpose(sections_abs_logits, perm=[1, 0, 2]),
                                                 sequence_length=sequence_lengths,
                                                 beam_width=3)
  return tf.sparse.to_dense(decoded[0])

def format_partial_abs(sections_abs_ids):
  result = []
  for abst in sections_abs_ids:
    if np.array(abst == 0).all():
      continue
    if abst[-1] == 0:
      abst = abst[:-1]

    result.append(abst)

  return result

In [ ]:
batch_num = 1 # номер батча в датасете
doc_num = 0 # номер примера в батче batch_num

test_arXiv = make_ds(vectorizer, "test.txt", max_doc_len=max_doc_len, max_abst_len=max_abst_len, max_sent_len=max_sent_len, batch_size=batch_size)
test_text = tf.data.TextLineDataset('test.txt').map((lambda x: tf.py_function(preproc_text_ds, [x], [tf.string]))).batch(batch_size)
test_sections = tf.data.TextLineDataset('test.txt').map((lambda x: tf.py_function(get_sections, [x], [tf.int64])))
test_text_sections = tf.data.TextLineDataset('test.txt').map((lambda x: tf.py_function(get_sections_text, [x], [tf.string])))


# генерация выхода модели
for doc, gt_lbels in test_arXiv.take(1):
  doc_ids = doc[doc_num]

  y_pred = imported_model.predict(doc)
  y_true = gt_lbels

logits = tf.math.log(y_pred)
sequence_lengths = tf.fill([batch_size], max_abst_len)
decoded, probs = tf.nn.ctc_beam_search_decoder(tf.transpose(logits, perm=[1, 0, 2]),
                                               sequence_length=sequence_lengths,
                                               beam_width=8)

decoded = tf.sparse.to_dense(decoded[0])

# вывод извлеченных предложений
for example in test_text.take(1):
  doc = example[0][doc_num]

  extracted_sentences = doc[decoded[doc_num]]
  extracted_sentences_ids = doc_ids[decoded[doc_num]]

  extracted_gt_summary = doc[gt_lbels[doc_num]]
  extracted_gt_summary_ids = doc_ids[gt_lbels[doc_num]]

print("Исходный текст:")
print(doc)
print()
print("Эталонная извлеченная аннотация:")
print(extracted_gt_summary)
print()
print("Извлеченные предложения:")
print(extracted_sentences)
print()
print("Индексы извлеченных предложений:")
print(decoded[doc_num])
print()

for example in test_sections.take(doc_num+1):
  example = example[0]
  extracted_ids = genereate_partial_abs(example, imported_model)

  extracted_ids = format_partial_abs(extracted_ids)

  for sections in test_text_sections.take(doc_num+1):
    sections = sections[0]

  result = []
  result_ids = []

  for sect_num in range(len(extracted_ids)):
    section_text = sections[sect_num]
    extracted_sect_sents = section_text[extracted_ids[sect_num]]
    extracted_sect_sents_ids = doc_ids[extracted_ids[sect_num]]

    result.append(extracted_sect_sents)
    result_ids.append(extracted_sect_sents_ids)

print("Извлеченные предложения разделов:")
print(result)

Исходный текст:
tf.Tensor(
[b'for about 20 years the problem of properties of short - term changes of solar activity has been considered extensively .'
 b'many investigators studied the short - term periodicities of the various indices of solar activity .'
 b'several periodicities were detected , but the periodicities about 155 days and from the interval of @xmath3 $ ] days ( @xmath4 $ ] years ) are mentioned most often .'
 b'first of them was discovered by @xcite in the occurence rate of gamma - ray flares detected by the gamma - ray spectrometer aboard the _ solar maximum mission ( smm ) .'
 b'this periodicity was confirmed for other solar flares data and for the same time period @xcite .'
 b'it was also found in proton flares during solar cycles 19 and 20 @xcite , but it was not found in the solar flares data during solar cycles 22 @xcite .'
 b'_    several autors confirmed above results for the daily sunspot area data . @xcite studied the sunspot data from 18741984 .'
 b'she found 

In [ ]:
sect_abs_text = np.concatenate(result)
extracted_sentences_ids = np.concatenate(extracted_sentences_ids)
extracted_gt_summary_ids = np.concatenate(extracted_gt_summary_ids)
result_ids = np.concatenate(result_ids)
result_ids = np.concatenate(result_ids)

references = tf.RaggedTensor.from_tensor([extracted_gt_summary_ids])

hypothesis = tf.RaggedTensor.from_tensor([result_ids])
f, p, r = text.metrics.rouge_l(hypothesis, references)
print("Качество аннотирования по частям: ")
print(f, p, r)
print()

hypothesis = tf.RaggedTensor.from_tensor([extracted_sentences_ids])
f, p, r = text.metrics.rouge_l(hypothesis, references)
print("Качество аннотирования целиком: ")
print(f, p, r)

Качество аннотирования по частям: 
tf.Tensor([0.09393939], shape=(1,), dtype=float32) tf.Tensor([0.14761905], shape=(1,), dtype=float32) tf.Tensor([0.06888889], shape=(1,), dtype=float32)

Качество аннотирования целиком: 
tf.Tensor([0.22280702], shape=(1,), dtype=float32) tf.Tensor([0.52916664], shape=(1,), dtype=float32) tf.Tensor([0.1411111], shape=(1,), dtype=float32)
